# Loading data

In [54]:
geno <-read.table("./Hetero_realigned_cov10_filtered3.raw", row.names=1, header=T)
row.names(geno)=1:240
x <- as.matrix(geno)-1   
pheno <- read.csv("./2017heteroPheno.csv", header=T)
attach(pheno)

The following objects are masked from pheno (pos = 8):

    bcw, gill, i5, i7, length, no, tank, vili, X



# I don not know how to estimate the SE or CI for heritability using rrBLUP

In [55]:
library(rrBLUP)
packageVersion("rrBLUP")

[1] '4.6'

In [56]:
A <- A.mat(x, n.core=8)
data <- data.frame(bcw=sqrt(pheno$bcw+1),tank=pheno$tank,length=pheno$length,gid=1:240)
row.names(A)=1:240;colnames(A)=1:240
ans_bcw <- kin.blup(data,geno="gid", pheno="bcw",K=A)
ans_length <- kin.blup(data,geno="gid",pheno="length",K = A)
h2_bcw <- ans_bcw$Vg/(ans_bcw$Vg+ans_bcw$Ve)        
h2_length <- ans_length$Vg/(ans_length$Vg+ans_length$Ve)
print(paste("h2_bcw", h2_bcw))
print(paste("h2_length", h2_length))

[1] "h2_bcw 0.6464662464747"
[1] "h2_length 0.843382207015855"


In [57]:
Bootstrap 95% CI for heritability
library(boot)

ERROR: Error in parse(text = x, srcfile = src): <text>:1:11: unexpected numeric constant
1: Bootstrap 95
              ^


In [18]:
data <- data.frame(bcw=sqrt(pheno$bcw+1),length=pheno$length,gid=1:240,G=x)

In [45]:
library(boot)
h2<-function(data,indices){
    d <- data[indices,]
    s<-data[indices,4:ncol(data)]
    m <- A.mat(s, n.core=8)
    ans_bcw <- kin.blup(d, geno="gid", pheno="bcw",K=m)
    #return(cor(d$bcw,ans_bcw$pred))
    h2_bcw <- ans_bcw$Vg/(ans_bcw$Vg+ans_bcw$Ve)
    return(h2_bcw)
}
results<-boot(data=data, statistic = h2,R=10)
boot.ci(results)

[1] "All values of t are equal to  0.999931804547672 \n Cannot calculate confidence intervals"


NULL

In [43]:
results$t

1
1
1
1
1
1
1
1
1
1


In [ ]:
par(mfrow=c(2,2))
#nf <- layout(matrix(c(1,2),ncol=2), widths=c(2,1), heights=c(2,2), TRUE)
hist(bcw,nclass=50,col="blue",ylim=c(0,15),main = "Histogram of HC",xlab = "H.okamotoi count")
qqnorm(bcw);qqline(bcw)
hist(sqrt(bcw+1),nclass=50,col="blue",ylim=c(0,15),cex.axis = 1.5, main= "Histogram of transformed data",
    xlab = "Transformed data")
qqnorm(sqrt(bcw+1));qqline(sqrt(bcw+1))

In [264]:
h2(data,sample(1:240,replace=TRUE))

[1] 0.9999318

In [19]:
indices<-(sample(1:240,replace=TRUE))

In [20]:
d <- data[indices,]
s<-data[indices,4:ncol(data)]
m <- A.mat(s, n.core=8)
ans_bcw <- kin.blup(d, geno="gid", pheno="bcw",K=m)
(h2_bcw <- ans_bcw$Vg/(ans_bcw$Vg+ans_bcw$Ve))

[1] 0.9999318

In [26]:
ans_bcw$Vg

[1] 2.560428

In [22]:
ans_bcw$Ve

[1] 0.0001746215

In [156]:
zoombot = boot.array(results, indices = T)

In [158]:
data[zoombot[4,],3]

[1] 192 228  77  52  45  17 216 229 216 234  21 142 159 151  26  20 143  71
 [19]  20 215 206 137 193   6 228  36 167  38 116  72  17 138 166  78 140  78
 [37] 117  63 226  55 122   8 233  46 134 173 224 186 102 148  95  75 192 176
 [55]  39  53 127 199  28  54 116 193 178 229  48 187  63 215 236 164  72  60
 [73] 215  72  66  29 217 236  33 184  39 121 173 204  41 240 177   8 207 187
 [91] 164 128 218 159 155 236  85 102  27  97  83  15  72   8 153  86 148  45
[109] 163  95 149 143 183 234 104  67 125  19  50 180  17 183  46  15 135 231
[127]  22   9  85 167 132 232 203 157 172 137 130 236  42  26  19 156 170 191
[145]  22  10  26  97  34  11 202  79 206 108  32  55  25  19 162 239 110 195
[163] 214 229 120  87  60  14 147  45 107  73  53 103  69 161  60  70 174 211
[181] 154 134 211 199 165 128 155 211  42 156 157 172 195 104 115 154 139  75
[199]   1  19  49 181  24 108 176 193  23 223  33 221 219  11  61 194 122 149
[217] 193 102 151 163 105 169  17  76  94 206 143 240 151  38  83 180  75   2
[235] 197   6 215 148 145 168

In [153]:
h2(data,zoombot[10,])
sort(data[zoombot[1,],3])

[1] 0.9999318

[1]   1   1   1   1   2   2   3   7   8   8   8   8   9   9  10  10  10  11
 [19]  14  15  16  16  17  19  21  21  23  24  25  27  28  29  29  31  32  35
 [37]  35  36  37  38  39  43  44  46  48  48  51  51  51  52  52  53  54  54
 [55]  55  55  56  57  58  60  62  63  65  66  66  66  66  67  67  68  70  70
 [73]  80  82  82  83  85  85  85  86  87  88  90  92  92  93  94  95  97 100
 [91] 101 101 104 105 106 107 107 107 109 109 111 111 113 113 113 114 114 115
[109] 115 115 116 116 116 116 117 121 124 124 124 125 126 126 128 128 130 130
[127] 130 135 137 137 138 141 142 143 144 145 146 146 147 147 150 151 151 151
[145] 151 152 154 157 159 159 160 162 163 164 164 165 165 168 169 169 169 171
[163] 171 171 173 173 174 175 176 177 177 177 178 178 179 180 180 180 181 182
[181] 182 182 182 185 186 187 188 188 189 190 190 192 194 198 199 200 201 203
[199] 205 205 206 206 206 207 208 209 209 209 210 211 213 213 216 216 217 218
[217] 220 220 222 224 224 226 227 227 227 228 229 229 231 232 232 235 237 237
[235] 237 237 237 238 240 240

In [68]:
plot(results)

[1] "All values of t* are equal to  0.653964961842951"


In [45]:
# function to obtain R-Squared from the data 
rsq <- function(formula, data, indices) {
  d <- data[indices,] # allows boot to select sample 
  fit <- lm(formula, data=d)
  return(summary(fit)$r.square)
} 
# bootstrapping with 1000 replications 
results <- boot(data=data, statistic=rsq, 
   R=1000, formula=mpg~wt+disp)

# view results
results 
plot(results)

# get 95% confidence interval 
boot.ci(results, type="bca")

ERROR: Error in eval(predvars, data, env): object 'mpg' not found


# I found the "heritability" package (Willem Kruijer et al, 2014), but result shows the lower heritability of resistance, 0.399 and the 95%CI is from 0.156 to 0.703.

In [58]:
library("heritability")

In [59]:
set.seed(12444) 
h2_bcw_conf <- marker_h2(data.vector=sqrt(pheno$bcw+1),geno.vector=1:240,K=as.matrix(A))
h2_length_conf <- marker_h2(data.vector=sqrt(pheno$length+1),geno.vector=1:240,K=as.matrix(A))

Iteration:  1 	 0.5165961 0.7986592 
Iteration:  2 	 0.514068 0.8021365 
Iteration:  3 	 0.514263 0.801985 
Iteration:  1 	 0.006931245 0.004870639 
Iteration:  2 	 0.00847863 0.004706491 
Iteration:  3 	 0.008777108 0.00460765 


In [60]:
print(paste("h2_bcw", h2_bcw_conf$h2,"h2_bcw_conf_95",h2_bcw_conf$conf.int2[1],h2_bcw_conf$conf.int2[2]))
print(paste("h2_length", h2_length_conf$h2,"h2_length_conf_95",h2_length_conf$conf.int2[1],h2_length_conf$conf.int2[2]))

[1] "h2_bcw 0.390703730038797 h2_bcw_conf_95 0.150057837829546 0.699609142184575"
[1] "h2_length 0.655753952212823 h2_length_conf_95 0.321755371654208 0.884379650808637"


# I only change the kinship matrix and re-estimate. Result almost the same.

In [61]:
library("synbreed")

In [62]:
m=data.frame(pheno=pheno$bcw)
lin_map<-read.table("./raw.map")
colnames(lin_map)<-c("chr","pos")
rownames(lin_map)<-colnames(x)
gp<-create.gpData(pheno=m,geno=x,map=lin_map)
#change into realized matrix (vanRaden, 2008)
re<-kin(codeGeno(gp),ret="realized")
l<-data.frame(re)
colnames(l)<-1:240
row.names(l)<-1:240

Warning message in df.allele$alter[alterMiss] <- unlist(multiLapply(as.data.frame(t(df.allele[alterMiss, :
"number of items to replace is not a multiple of replacement length"

In [63]:
set.seed(12444) 
h2_bcw_conf <- marker_h2(data.vector=pheno$bcw,geno.vector=1:240,K=as.matrix(l))
h2_length_conf <- marker_h2(data.vector=pheno$length,geno.vector=1:240,K=as.matrix(l))

Iteration:  1 	 6.03554 77.96402 
Iteration:  2 	 21.98713 56.77749 
Iteration:  3 	 31.57139 51.30268 
Iteration:  4 	 32.87825 50.70687 
Iteration:  5 	 32.7939 50.78098 
Iteration:  6 	 32.80272 50.77397 
Iteration:  7 	 32.80184 50.77468 
Iteration:  8 	 32.80193 50.77461 
Iteration:  1 	 0.3052749 0.2136605 
Iteration:  2 	 0.3416217 0.225045 
Iteration:  3 	 0.3453297 0.225978 
Iteration:  4 	 0.3452718 0.2260502 


In [64]:
print(paste("h2_bcw", h2_bcw_conf$h2,"h2_bcw_conf_95",h2_bcw_conf$conf.int2[1],h2_bcw_conf$conf.int2[2]))
print(paste("h2_length", h2_length_conf$h2,"h2_length_conf_95",h2_length_conf$conf.int2[1],h2_length_conf$conf.int2[2]))

[1] "h2_bcw 0.392477732103251 h2_bcw_conf_95 0.155101177244083 0.694516715547012"
[1] "h2_length 0.604338275580889 h2_length_conf_95 0.298544472493573 0.845715871772032"


# BGLR

In [78]:
library(BGLR)

In [79]:
fm=BGLR(y=pheno$bcw,ETA=list(list(X=geno,model='BRR',saveEffects=T)),nIter=500,burnIn=200,verbose=F)
 varU=scan('ETA_1_varB.dat')
 varE=scan('varE.dat')
 h2=varU/(varU+varE)


In [80]:
cor(pheno$bcw,fm$yHat)

[1] 0.8581241

In [81]:
mean(varE)

[1] 59.74615

In [82]:
mean(fm$varE)

[1] 59.43138

In [83]:
fm$ETA[[1]]$varB

[1] 0.04150122